In [1]:
import numpy as np

In [2]:
def nn(x, w): return np.sign(np.dot(x, w)).astype(int)

In [3]:
x = np.array([[-2,4,-1], [4,1,-1], [1, 6, -1], [2, 4, -1], [6, 2, -1]])
t = np.array([-1, -1, 1, 1, 1])

In [4]:
n_samples, n_features = x.shape

In [5]:
w = np.zeros(n_features)

In [6]:
w

array([0., 0., 0.])

In [7]:
nn(x, w)

array([0, 0, 0, 0, 0])

In [8]:
np.sign([0, 10, -11])

array([ 0,  1, -1])

In [9]:
ld = 1 / 1000

In [48]:
def cost(y, t, w, ld): return np.sum(np.fmax(1 - np.multiply(t, nn(x, w)), 0).reshape(-1, 1) + np.sum(ld * w ** 2, 0))

In [33]:
def gradient(w, x, t, ld): 
  return np.sum(np.multiply(t.reshape(-1,1), x) * np.fmax(np.sign(1 - np.multiply(t, np.dot(x, w))), 0).reshape(-1,1), 0) + (ld * 2 * w)

In [12]:
def delta_w(w, x, t, learning_rate, ld):
    return learning_rate * gradient(w, x, t, ld)

In [36]:
learning_rate = 0.1; nb_of_iterations = 20

In [49]:
w = np.zeros(n_features)
w_cost = [(w, w, cost(nn(x, w), t, w, ld))]
for i in range(nb_of_iterations):
    w = w - delta_w(w, x, t, learning_rate, ld)
    w_cost.append((w, delta_w(w, x, t, learning_rate, ld), cost(nn(x, w), t, w, ld)))
w_cost

[(array([0., 0., 0.]), array([0., 0., 0.]), 5.0),
 (array([-0.7, -0.7,  0.1]),
  array([ 0.89986,  1.19986, -0.29998]),
  6.004949999999999),
 (array([-1.59986, -1.89986,  0.39998]),
  array([ 0.89968003,  1.19962003, -0.29992   ]),
  6.031645020197999),
 (array([-2.49954003, -3.09948003,  0.6999    ]),
  array([ 0.89950009,  1.1993801 , -0.29986002]),
  6.081721684055722),
 (array([-3.39904012, -4.29886013,  0.99976002]),
  array([ 0.89932019,  1.19914023, -0.29980005]),
  6.155165961386848),
 (array([-4.29836031, -5.49800036,  1.29956007]),
  array([ 0.89914033,  1.1989004 , -0.29974009]),
  6.251963828552264),
 (array([-5.19750064, -6.69690076,  1.59930016]),
  array([ 0.8989605 ,  1.19866062, -0.29968014]),
  6.372101268457259),
 (array([-6.09646114, -7.89556138,  1.8989803 ]),
  array([ 0.89878071,  1.19842089, -0.2996202 ]),
  6.515564270548721),
 (array([-6.99524185, -9.09398227,  2.1986005 ]),
  array([ 0.89860095,  1.1981812 , -0.29956028]),
  6.682338830812322),
 (array([ -7.

In [16]:
t

array([-1, -1,  1,  1,  1])

In [17]:
nn(x, w)

array([-1, -1, -1, -1, -1])

In [18]:
np.multiply(t, nn(x, w))

array([ 1,  1, -1, -1, -1])

In [19]:
np.fmax(1 - np.multiply(t, nn(x, w)), 0)

array([0, 0, 2, 2, 2])

In [20]:
w

array([-17.73373539, -23.43373539,   5.86626461])

In [21]:
w ** 2

array([314.48537081, 549.13995424,  34.4130605 ])

In [22]:
cost(nn(x, w), t, w, ld)

6.898038385550921

In [23]:
np.fmax(np.sign(1 - np.multiply(t, np.dot(x, w))), 0) 

array([0., 0., 1., 1., 1.])

In [24]:
np.sum(np.multiply(t.reshape(-1,1), x), 0)

array([ 7,  7, -1])

In [25]:
t

array([-1, -1,  1,  1,  1])

In [26]:
t.T

array([-1, -1,  1,  1,  1])

In [27]:
t == t.T

array([ True,  True,  True,  True,  True])

In [28]:
np.sum(np.multiply(t.reshape(-1,1), x) * np.fmax(np.sign(1 - np.multiply(t, np.dot(x, w))), 0).reshape(-1,1), 0) 

array([ 9., 12., -3.])

In [47]:
np.sum(np.fmax(1 - np.multiply(t, nn(x, w)), 0).reshape(-1, 1) + np.sum(ld * w ** 2, 0))

5.0